# BTSnoop Log Sniffer

This notebook demonstrates the process of parsing bluetooth packet data from android hci logs.

In [1]:
# !pip install fpdf
# !pip install kaleido
# !pip install dataframe_image

In [2]:
import sys
import json
import requests


import pandas as pd
from fpdf import FPDF
from pprint import pprint
from decimal import Decimal
import dataframe_image as dfi
from datetime import datetime

# import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import fpdf

from constants import *


pd.set_option('display.max_columns', None)

# filepath = sys.argv[1]

# with open(filepath, 'r') as file:
with open('./output1.json', 'r') as file:
    data = [x["_source"]["layers"] for x in json.load(file)]

# TODO: SQLite storage for known devices

[source](https://github.com/nccgroup/BLE-Replay/blob/master/btsnoop/btsnoop/bt/hci_evt.py)

The HCI LE Meta Event is used to encapsulate all LE Controller specific events.
The Event Code of all LE Meta Events shall be 0x3E. The Subevent_Code is
the first octet of the event parameters. The Subevent_Code shall be set to one
of the valid Subevent_Codes from an LE specific event


HCI inherently cannot differentiate between packet types. Hence a common physical interface is used with the indicators that are sent right before the packet is sent. These indicators are as follows

| HCI Packet Type | HCI Packet Indicator |
| --------------- | -------------------- |
| HCI Command Packet | 0x01 |
| HCI ACL Data Packet | 0x02 |
| HCI Synchronous Data Packet | 0x03 |
| HCI Event Packet | 0x04 |
| HCI ISO Data Packet | 0x05 |

LE Meta events encapsulate all LE Controller events and have a code of 0x03

protocols = {
    "bluetooth:hci_h4:bthci_acl:btl2cap": 'ACL-L2CAP Event',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btatt": 'ACL-L2CAP - ATT (Attribute) Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btavctp:btavrcp": 'ACL-L2CAP - AVCTP/AVR Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btavdtp": 'ACL-L2CAP - AVDTP Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btrfcomm": 'ACL-L2CAP - RF Communication protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btrfcomm:bthfp": 'ACL-L2CAP - Hands Free Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btrfcomm:data": 'ACL-L2CAP - Data Transfer',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btsdp": 'ACL-L2CAP - Service Discovery Protocol (SDP)',
    "bluetooth:hci_h4:bthci_cmd": 'HCI Command',
    "bluetooth:hci_h4:bthci_cmd:btcommon": 'Bluetooth Advertising Event',
    "bluetooth:hci_h4:bthci_cmd:bthci_vendor.broadcom": 'Vendor-Specific Broadcom Packet',
    "bluetooth:hci_h4:bthci_evt": 'HCI Event',
    "bluetooth:hci_h4:bthci_evt:btcommon": 'Bluetooth Advertising Event',
    "bluetooth:hci_h4:bthci_evt:bthci_vendor.broadcom": 'Vendor-Specific Broadcom Packet'
    }

In [ ]:

def format_packet_data(data):
    data_input = data
    for packet in data_input:
        packet["Sequence Number"] = packet["frame.number"][0] if "frame.number" in packet else None
        packet["Epoch Timestamp"] = packet["frame.time_epoch"][0] if "frame.time_epoch" in packet else None
        packet["Timestamp"] = datetime.fromtimestamp(int(Decimal(packet["frame.time_epoch"][0]))) if "frame.time_epoch" in packet else None
        packet["Packet Length"] = packet["frame.len"][0] if "frame.len" in packet else None
        packet["Protocol"] = packet["frame.protocols"][0] if "frame.protocols" in packet else None
        packet["LE Event Code"] = packet["hci_h4.type"][0] if "hci_h4.type" in packet else None
        packet["LE Event Type"] = HCI_LE_EVENT[packet["hci_h4.type"][0]] if "hci_h4.type" in packet else None
        packet["Direction"] = EVENT_DIRECTION[packet["hci_h4.direction"][0]] if "hci_h4.direction" in packet else None
        packet["HCI Command"] = packet["bthci_cmd"][0] if "bthci_cmd" in packet else None
        packet["HCI Event Code"] = packet["bthci_evt.code"][0] if "bthci_evt.code" in packet else None
        packet["HCI Event"] = packet["bthci_evt"][0] if "bthci_evt" in packet else None
        packet["HCI Event Command Packet Count"] = packet["bthci_evt.num_command_packets"][0] if "bthci_evt.num_command_packets" in packet else None
        packet["HCI Event Status"] = packet["bthci_evt.status"][0] if "bthci_evt.status" in packet else None
        packet["HCI Event Command in Frame"] = packet["bthci_evt.command_in_frame"][0] if "bthci_evt.command_in_frame" in packet else None
        packet["HCI ACL chandle"] = packet["bthci_acl.chandle"][0] if "bthci_acl.chandle" in packet else None
        packet["HCI ACL Length"] = packet["bthci_acl.length"][0] if "bthci_acl.length" in packet else None
        packet["Source Device MAC"] = packet["bthci_acl.src.bd_addr"][0] if "bthci_acl.src.bd_addr" in packet else None
        packet["Source Device Name"] = packet["bthci_cmd.device_name"][0] if "bthci_cmd.device_name" in packet else (packet["bthci_acl.src.name"][0] if "bthci_acl.src.name" in packet else "Unknown Device")
        packet["Destination Device MAC"] = packet["bthci_acl.dst.bd_addr"][0] if "bthci_acl.dst.bd_addr" in packet else None
        packet["Destination Device Name"] = packet["bthci_acl.dst.name"][0] if "bthci_acl.dst.name" in packet else None
        packet["L2CAP CID"] = packet["btl2cap.cid"][0] if "btl2cap.cid" in packet else None
        packet["L2CAP Length"] = packet["btl2cap.length"][0] if "btl2cap.length" in packet else None


        if packet["Source Device Name"] == "":
            packet["Source Device Name"] = f"Unknown Device-{packet['Source Device MAC'][:8]}" if packet["Source Device MAC"] else "Unknown Device"

        if packet["Destination Device Name"] == "":
            packet["Destination Device Name"] = f"Unknown Device-{packet['Destination Device MAC'][:8]}" if packet["Destination Device MAC"] else "Unknown Device"


        if packet["L2CAP CID"] is not None:
            if packet["L2CAP CID"] in L2CAP_CID_VALUES:
                packet["L2CAP Type"] = L2CAP_CID_VALUES[packet["L2CAP CID"]]
            else:
                packet["L2CAP Type"] = 'Dynamically Allocated'

        # Deleting Original BT PCAP keys
        for key in PACKET_KEYS:
            if key in packet:
                del packet[key]

        # Deleting None Values
        keys_to_delete = [key for key, value in packet.items() if value is None]
        for key in keys_to_delete:
            del packet[key]
    
    return pd.DataFrame(data)

In [4]:
# df_data = pd.DataFrame(data)

df_data = format_packet_data(data)

In [5]:
def generate_device_piecharts(df, names, title, export_filename):
    fig = px.pie(df, names=names, title=title, hole=0.3, width=750, height=400)
    fig.update_layout(title_font_size=24)
    fig.update_traces(textinfo='percent', texttemplate='%{percent:.2%}', textfont_size=18)
    fig.update_layout(legend=dict(x=0.85, y=1.1))
    fig.update_layout(margin=dict(t=50, b=30, l=0, r=220))
    fig.write_image(f'./generated_images/{export_filename}', scale=2)
    # fig.show()

In [6]:
source_mac_info = df_data['Source Device Name'] + "\n(" + df_data['Source Device MAC'] + ")"
source_mac_info = source_mac_info.dropna()
df_sources = pd.DataFrame(source_mac_info, columns=['Source Devices'])

generate_device_piecharts(df_sources, 'Source Devices', 'Distribution of Source Devices', 'source_devices_distribution.png')

In [7]:
destination_device_info = df_data['Destination Device Name'] + "\n(" + df_data['Destination Device MAC'] + ")"
destination_device_info = destination_device_info.dropna()
df_destinations = pd.DataFrame(destination_device_info, columns=['Destination Devices'])

generate_device_piecharts(df_destinations, 'Destination Devices', 'Distribution of Destination Devices', 'destination_devices_distribution.png')

In [8]:
def generate_hci_cmdevt_plot(df_hci_cmd, df_hci_event, export_filename):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(
            x=df_hci_cmd['Timestamp'],
            y=df_hci_cmd['Sequence Number'],
            mode='lines+markers',
            name='HCI Commands',
            line=dict(color='red')),
            secondary_y=False
        )

    fig.add_trace(
        go.Scatter(
            x=df_hci_event['Timestamp'],
            y=df_hci_event['Sequence Number'],
            mode='lines+markers',
            name='HCI Events',
            line=dict(color='blue')),
            secondary_y=True
        )

    fig.update_yaxes(title_text="Sequence Number (HCI Commands)", dtick=200, tickmode='linear', secondary_y=False)
    fig.update_yaxes(title_text="Sequence Number (HCI Events)", dtick=200, tickmode='linear', secondary_y=True)

    fig.update_layout(title='HCI Events and Commands over Time',
                      title_font_size=24,
                      xaxis_title='Timestamp (UTC)',
                      xaxis=dict(tickangle=45),
                      legend=dict(x=0.01, y=0.99),
                      template='plotly_white',
                      width=900,
                      height=600)
    fig.write_image(f'./generated_images/{export_filename}', scale=2)
    # fig.show()

In [9]:
df_hci_cmd = df_data[df_data['LE Event Type'] == 'Bluetooth HCI Command (HCI_CMD)'].dropna(axis=1, how='all')
df_hci_event = df_data[df_data['LE Event Type'] == 'Bluetooth HCI Event (HCI_EVT)'].dropna(axis=1, how='all')
df_hci_event_filtered = df_hci_event[df_hci_event['HCI Event Command in Frame'].isin(df_hci_cmd['Sequence Number'])]

generate_hci_cmdevt_plot(df_hci_cmd, df_hci_event_filtered, export_filename="hci_cmd_event_communication.png")

In [10]:
unique_src_macs = df_data['Source Device MAC'].dropna().unique()
unique_dst_macs = df_data['Destination Device MAC'].dropna().unique()

print("Unique Source MAC Addresses:")
print(unique_src_macs)

print("\nUnique Destination MAC Addresses:")
print(unique_dst_macs)

Unique Source MAC Addresses:
['d0:62:2c:59:12:92' 'd4:3a:2c:a9:32:4f' 'b4:84:d5:99:b9:f3']

Unique Destination MAC Addresses:
['d4:3a:2c:a9:32:4f' 'd0:62:2c:59:12:92' 'b4:84:d5:99:b9:f3']


In [11]:
def get_mac_info(mac_address):
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    url = VENDOR_LOOKUP_API + mac_address
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()[0]
    else:
        return None

def generate_mac_info_dataframe(unique_mac_list):
    unique_mac_info_list = []
    for mac in unique_mac_list:
        info = get_mac_info(mac)
        if info:
            unique_mac_info_list.append(info)
    mac_info_df = pd.DataFrame(unique_mac_info_list)[['company', 'addressL1', 'addressL2', 'addressL3', 'country']]
    return unique_mac_info_list

In [12]:

df_src_mac_info = generate_mac_info_dataframe(unique_src_macs.tolist())
df_dst_mac_info = generate_mac_info_dataframe(unique_dst_macs.tolist())

pprint(df_src_mac_info)
pprint(df_dst_mac_info)

[{'addressL1': 'Floor 5, Block C, Huanpu Industrial Park, 211 Tiangu 8th Road',
  'addressL2': "Xi 'an   Shaanxi  710076",
  'addressL3': '',
  'company': "Xi'an Yipu Telecom Technology Co.,Ltd.",
  'country': 'CN',
  'endDec': '229120080347135',
  'endHex': 'D0622CFFFFFF',
  'startDec': '229120063569920',
  'startHex': 'D0622C000000',
  'type': 'MA-L'},
 {'addressL1': '1600 Amphitheatre Parkway',
  'addressL2': 'Mountain View  CA  94043',
  'addressL3': '',
  'company': 'Google, Inc.',
  'country': 'US',
  'endDec': '233346328166399',
  'endHex': 'D43A2CFFFFFF',
  'startDec': '233346311389184',
  'startHex': 'D43A2C000000',
  'type': 'MA-L'},
 {'addressL1': 'RM1601,Crative BuildingII East Tianan',
  'addressL2': 'City Futian Shenzhen  Guangdong  518000',
  'addressL3': '',
  'company': 'GooWi Wireless Technology Co., Limited',
  'country': 'CN',
  'endDec': '198482619006975',
  'endHex': 'B484D5FFFFFF',
  'startDec': '198482602229760',
  'startHex': 'B484D5000000',
  'type': 'MA-L'}]


In [13]:
df_acl_le_events = df_data[(df_data['LE Event Type'] == 'Bluetooth HCI ACL (HCI_ACL)') & (df_data['Protocol'].str.contains('bthci_acl'))]

In [14]:
df_acl_le_events.head()

,Sequence Number,Epoch Timestamp,Timestamp,Packet Length,Protocol,LE Event Code,LE Event Type,Direction,HCI Command,Source Device Name,HCI Event Code,HCI Event,HCI Event Command Packet Count,HCI Event Status,HCI Event Command in Frame,btatt.opcode,HCI ACL chandle,HCI ACL Length,Source Device MAC,Destination Device MAC,Destination Device Name,L2CAP CID,L2CAP Length,L2CAP Type,btatt.value
144,145,1727925136.373296000,2024-10-02 23:12:16,12,bluetooth:hci_h4:bthci_acl:btl2cap:btatt,0x02,Bluetooth HCI ACL (HCI_ACL),Controller > Host,NaN,Unknown Device-d0:62:2c,NaN,NaN,NaN,NaN,NaN,[0x02],0x0040,7,d0:62:2c:59:12:92,d4:3a:2c:a9:32:4f,Pixel 7 Pro,0x0004,3,Attribute Protocol (ATT),NaN
145,146,1727925136.374463000,2024-10-02 23:12:16,12,bluetooth:hci_h4:bthci_acl:btl2cap:btatt,0x02,Bluetooth HCI ACL (HCI_ACL),Host > Controller,NaN,Pixel 7 Pro,NaN,NaN,NaN,NaN,NaN,[0x03],0x0040,7,d4:3a:2c:a9:32:4f,d0:62:2c:59:12:92,Unknown Device-d0:62:2c,0x0004,3,Attribute Protocol (ATT),NaN
153,154,1727925136.420924000,2024-10-02 23:12:16,16,bluetooth:hci_h4:bthci_acl:btl2cap:btatt,0x02,Bluetooth HCI ACL (HCI_ACL),Host > Controller,NaN,Pixel 7 Pro,NaN,NaN,NaN,NaN,NaN,[0x08],0x0040,11,d4:3a:2c:a9:32:4f,d0:62:2c:59:12:92,Unknown Device-d0:62:2c,0x0004,7,Attribute Protocol (ATT),NaN
208,209,1727925136.508358000,2024-10-02 23:12:16,14,bluetooth:hci_h4:bthci_acl:btl2cap:btatt,0x02,Bluetooth HCI ACL (HCI_ACL),Controller > Host,NaN,Unknown Device-d0:62:2c,NaN,NaN,NaN,NaN,NaN,[0x01],0x0040,9,d0:62:2c:59:12:92,d4:3a:2c:a9:32:4f,Pixel 7 Pro,0x0004,5,Attribute Protocol (ATT),NaN
210,211,1727925136.509110000,2024-10-02 23:12:16,16,bluetooth:hci_h4:bthci_acl:btl2cap:btatt,0x02,Bluetooth HCI ACL (HCI_ACL),Host > Controller,NaN,Pixel 7 Pro,NaN,NaN,NaN,NaN,NaN,[0x10],0x0040,11,d4:3a:2c:a9:32:4f,d0:62:2c:59:12:92,Unknown Device-d0:62:2c,0x0004,7,Attribute Protocol (ATT),NaN


The Bluetooth Asynchronous Connection-oriented Logical transport (ACL) is a method of transfer of asynchronous (data that is not in real time) data over a bluetooth connection.

Looking at some of the profiles under ACL we see the following:

- *AVDTP*: This is the Audio/Video Distribution Transport Protocol (AVDTP) signifies the streaming of music to a bluetooth speaker device such as a set of headsets over the L2CAP protocol.
- *HFP*: The hands free profile (HFP) can be seen as a mode of remote control between the phone and bluetooth device.
- *AVRCP*: The Audio/Video Remote Control Profile is used along with the A2DP (Advanced Audio Distribution Profile) profile to allow a single remote device to control a bluetooth device
- *SDP*: The Service Discovert Protocol is a manner of communication to discover the available bluetooth devices nearby and their types.

[Reference](https://datatracker.ietf.org/doc/html/rfc1761)

Android bluetooth logs come in the **Snoop Version 1 Packet Capture File Format** which is similar to the second version developed by Sun Microsystems in 1995. When we capture logs of this format, we obtain arrays of octets (8 bit packets of information), with each array item corresponding to a packet record.
